In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

#'CNNLSTMYelpMultiV2.h5'
#LSTMBYelpMultiV1.h5


print ( " Hello")
yelp = pd.read_csv('C:\\Users\\45058393\\Downloads\\yelp_reviewBig\\yelp_review.csv')


#RowTest= yelp[:100000]
#RoWTest2 = yelp[:38000]
#print(RowTest.shape)

#print(RowTest.head())

#RowTraining = yelp[200000:700000]
#print(RowTraining.shape)

#print(RowTraining.head())


reviews=yelp[:500000]    #----- change to 600000
#reviews=reviews[reviews.stars!=3]

#reviews["labels"]= reviews["stars"].apply(lambda x: 1 if x > 3  else 0)
#reviews=reviews.drop("stars",axis=1)
print(reviews.shape)
print(reviews.head())


print( "Hi")
texts = reviews["text"].values
labels = reviews["stars"].values

MAX_NUM_WORDS=1000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH=100 # max number of words in a review to use

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import utils as np_utils
from keras.utils.np_utils import to_categorical


tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
labels = to_categorical(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


VALIDATION_SPLIT=0.2

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print( " ok")


#GLOVE_DIR='../input/glove-global-vectors-for-word-representation/'
GLOVE_DIR='C:\\Users\\45058393\\Downloads\\glove.6B'

import os
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


EMBEDDING_DIM = 50 # how big is each word vector

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print ( " OK")



from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


from keras.layers import Bidirectional, GlobalMaxPool1D,Conv1D
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation

from keras.models import Model


inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedded_sequences = embedding_layer(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="softmax")(x)
modelLSTMB = Model(inputs=inp, outputs=x)
modelLSTMB.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


modelLSTMB.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=40, batch_size=128)

#modelLSTMB.save('modelLSTMBYelpMulti.h5')


modelLSTMB.save('LSTMBYelpMultiV1.h5')

reviewsTest=yelp[600000:650000]    #----- change to 600000

#reviewsTest=reviewsTest[reviewsTest.stars!=3]
#reviewsTest["labels"]= reviewsTest["stars"].apply(lambda x: 1 if x > 3  else 0)
#reviewsTest=reviewsTest.drop("stars",axis=1)

print(reviewsTest.head())
print(reviewsTest.shape)


textsTest = reviewsTest["text"].values
labelsTest = reviewsTest["stars"].values

sequencesTest = tokenizer.texts_to_sequences(textsTest)
dataTest = pad_sequences(sequencesTest, maxlen=MAX_SEQUENCE_LENGTH)
labelsTest = to_categorical(labelsTest)


result = modelLSTMB.evaluate(dataTest,labelsTest,verbose =0)
predication1 = modelLSTMB.predict(dataTest)
RA = predication1.argmax(axis = -1)
print(result)
print(" Acc %.2f%%" % (result[1]*100))
print(predication1)
print("------------")
print(RA)



 Hello
(500000, 9)
                review_id                 user_id             business_id  \
0  vkVSCC7xljjrAI4UGfnKEQ  bv2nCi5Qv5vroFiqKGopiw  AEx2SYEUJmTxVVB18LlCwA   
1  n6QzIUObkYshz4dz2QRJTw  bv2nCi5Qv5vroFiqKGopiw  VR6GpWIda3SfvPC-lg9H3w   
2  MV3CcKScW05u5LVfF6ok0g  bv2nCi5Qv5vroFiqKGopiw  CKC0-MOWMqoeWf6s-szl8g   
3  IXvOzsEMYtiJI0CARmj77Q  bv2nCi5Qv5vroFiqKGopiw  ACFtxLv8pGrrxMm6EgjreA   
4  L_9BTb55X0GDtThi6GlZ6w  bv2nCi5Qv5vroFiqKGopiw  s2I_Ni76bjJNK9yG60iD-Q   

   stars        date                                               text  \
0      5  2016-05-28  Super simple place but amazing nonetheless. It...   
1      5  2016-05-28  Small unassuming place that changes their menu...   
2      5  2016-05-28  Lester's is located in a beautiful neighborhoo...   
3      4  2016-05-28  Love coming here. Yes the place always needs t...   
4      4  2016-05-28  Had their chocolate almond croissant and it wa...   

   useful  funny  cool  
0       0      0     0  
1       0      0 

In [5]:
from keras.layers import Bidirectional, GlobalMaxPool1D,Conv1D
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation

from keras.models import Model



inp = Input(shape=(MAX_SEQUENCE_LENGTH,  ))

y = embedded_sequences = embedding_layer(inp)
#y = Embedding(MAX_SEQUENCE_LENGTH, , input_length=MAX_NUM_WORDS)(y)
print(y.shape)
#x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
y = Dropout(0.1)(y)
print(y.shape)
y = Conv1D(64, 5, activation='relu')(y)
print(y.shape)
#y = MaxPooling1D(pool_size=4)(y)
#y = GlobalMaxPool1D()(y)
y = LSTM(100)(y)
#y = LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(y)
print(y.shape)
#x = Dense(50, activation="relu")(x)
#x = Dropout(0.1)(x)
y = Dense(50, activation="relu")(y)
print(y.shape)
y = Dropout(0.1)(y)
print(y.shape)
y = Dense(6, activation="softmax")(y)
print(y.shape)

CNNLSTMModel = Model(inputs=inp, outputs=y)
CNNLSTMModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(x_train.shape)
print(y_train.shape)
#CNNLSTMModel.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2)  #, batch_size=128


CNNLSTMModel.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40 , batch_size=128)
CNNLSTMModel.save('CNNLSTMYelpMultiV2.h5')


result2 = CNNLSTMModel.evaluate(dataTest,labelsTest,verbose =0)
print(result2)
print(" Acc %.2f%%" % (result2[1]*100))

predication12 = modelLSTMB.predict(dataTest)
RA2 = predication1.argmax(axis = -1)
print(predication12)
print("------------")
print(RA2)

(?, 100, 50)
(?, 100, 50)
(?, 96, 64)
(?, 100)
(?, 50)
(?, 50)
(?, 6)
(400000, 100)
(400000, 6)
Train on 400000 samples, validate on 100000 samples
Epoch 1/40
400000/400000 [==============================] - 1028s 3ms/step - loss: 0.2867 - acc: 0.8737 - val_loss: 0.2574 - val_acc: 0.8844
Epoch 2/40
400000/400000 [==============================] - 795s 2ms/step - loss: 0.2600 - acc: 0.8833 - val_loss: 0.2516 - val_acc: 0.8857
Epoch 3/40
400000/400000 [==============================] - 798s 2ms/step - loss: 0.2523 - acc: 0.8861 - val_loss: 0.2451 - val_acc: 0.8887
Epoch 4/40
400000/400000 [==============================] - 804s 2ms/step - loss: 0.2471 - acc: 0.8884 - val_loss: 0.2418 - val_acc: 0.8899
Epoch 5/40
400000/400000 [==============================] - 809s 2ms/step - loss: 0.2429 - acc: 0.8899 - val_loss: 0.2415 - val_acc: 0.8903
Epoch 6/40
400000/400000 [==============================] - 809s 2ms/step - loss: 0.2398 - acc: 0.8912 - val_loss: 0.2363 - val_acc: 0.8927
Epoch 7/40


KeyboardInterrupt: 